# Load packages

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib as mpl
import yfinance as yf
import seaborn as sns
import warnings

from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.api import VAR

# Configuration

In [ ]:
# Korean font
from matplotlib import font_manager, rc
try:
    font_path = "C:/Windows/Fonts/malgun.TTF"
    Kfont = font_manager.FontProperties(fname=font_path).get_name()
    rc('font', family=Kfont)
except:
    pass

# Fix minus presentation
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
# Fix random seed
def fix_random_seed(seed=42):
    import random
    import numpy as np 
    import os

    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
fix_random_seed()

# Load preprocessed data

In [ ]:
df_core = pd.read_csv('data/preprocessed_data.csv')
df_core.rename(columns={'날짜': 'date'}, inplace=True)
df_core.set_index('date', inplace=True)
df_core

# Analysis with VAR model

In [ ]:
mdl_var = VAR(df_core)
var_max_lag = 6
best_lag = mdl_var.select_order(var_max_lag)
best_lag.summary()

In [ ]:
if best_lag.selected_orders['aic'] == 0:
    var_best_lag = var_max_lag
else:
    var_best_lag = best_lag.selected_orders['aic']
    
rslt_var = mdl_var.fit(var_best_lag)
rslt_var.summary()

In [ ]:
var_pval_tf_mat = rslt_var.pvalues.applymap(lambda x: True if x < 0.05 else False)
var_pval_tf_mat.head()

In [ ]:
# plot - VAR model coefficients p-value
fig_var_pval, ax_var_pval = plt.subplots(1, 1, figsize=(4*var_best_lag, 6*var_best_lag))

heat_pval = sns.heatmap(
    var_pval_tf_mat.T, 
    square=True, 
    ax=ax_var_pval, 
    annot=True,
    annot_kws={'fontsize':20-var_best_lag}, 
    cbar=False,
    linecolor='grey',
    linewidth=0.1,
    )
heat_pval.set_xticklabels(heat_pval.get_xticklabels(), fontsize=12)
heat_pval.set_yticklabels(heat_pval.get_yticklabels(), fontsize=12)
ax_var_pval.set_title('Statistical significances of VAR coefficients by p-values', fontsize=15, family='bold')

In [ ]:
# Analyze causality between two time series variables with Granger Causality Test
selc_granger_test = [ 
    # (feature x, feature y, time_lag)
    ('carbon_credit', 'natural_gas', 5),
    ('brent_crude_future', 'kospi', 1)
]

df_gct = pd.DataFrame(
    index=[x[0] for x in selc_granger_test]+[x[1] for x in selc_granger_test],
    columns=[x[0] for x in selc_granger_test]+[x[1] for x in selc_granger_test],
    dtype='object',
    )
df_gct = df_gct.applymap(lambda x: pd.Series(index=np.arange(1, var_max_lag+1), dtype='object'))
df_gct.index.name='cause'
df_gct.columns.name='effect'

for factor_x, factor_y, time_lag in selc_granger_test:
    # factor x -> factor y
    print(f'\n[{factor_x}][t-{time_lag}] -> [{factor_y}][t]', end='')
    df_gct.loc[factor_x, factor_y][time_lag] = grangercausalitytests(
        df_core[[factor_y, factor_x]],
        maxlag=[time_lag],
        verbose=1
    )

    # factor y -> factor x
    print(f'\n[{factor_y}][t-{time_lag}] -> [{factor_x}][t]', end='')
    df_gct.loc[factor_y, factor_x][time_lag] = grangercausalitytests(
        df_core[[factor_x, factor_y]],
        maxlag=[time_lag],
        verbose=1
        )


df_gct